<a href="https://colab.research.google.com/github/Dhaya1719/Airline-Ticket-Reservation-System/blob/main/sign_lang_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()   # Choose kaggle.json from your computer


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"freakyboy17","key":"61990927fd91d8cf105c101baf12eb05"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d risangbaskoro/wlasl-processed
!unzip wlasl-processed.zip -d wlasl_data


Streaming output truncated to the last 5000 lines.
  inflating: wlasl_data/videos/43596.mp4  
  inflating: wlasl_data/videos/43598.mp4  
  inflating: wlasl_data/videos/43599.mp4  
  inflating: wlasl_data/videos/43671.mp4  
  inflating: wlasl_data/videos/43672.mp4  
  inflating: wlasl_data/videos/43674.mp4  
  inflating: wlasl_data/videos/43677.mp4  
  inflating: wlasl_data/videos/43679.mp4  
  inflating: wlasl_data/videos/43680.mp4  
  inflating: wlasl_data/videos/43681.mp4  
  inflating: wlasl_data/videos/43682.mp4  
  inflating: wlasl_data/videos/43684.mp4  
  inflating: wlasl_data/videos/43689.mp4  
  inflating: wlasl_data/videos/43697.mp4  
  inflating: wlasl_data/videos/43698.mp4  
  inflating: wlasl_data/videos/43700.mp4  
  inflating: wlasl_data/videos/43703.mp4  
  inflating: wlasl_data/videos/43726.mp4  
  inflating: wlasl_data/videos/43727.mp4  
  inflating: wlasl_data/videos/43729.mp4  
  inflating: wlasl_data/videos/43730.mp4  
  inflating: wlasl_data/videos/43733.mp4  
  i

In [4]:
import json

with open("wlasl_data/WLASL_v0.3.json", "r") as f:
    wlasl_anno = json.load(f)

print("Samples:", len(wlasl_anno))
print("First entry:", wlasl_anno[0])


Samples: 2000
First entry: {'gloss': 'book', 'instances': [{'bbox': [385, 37, 885, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 0, 'signer_id': 118, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'variation_id': 0, 'video_id': '69241'}, {'bbox': [190, 25, 489, 370], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 1, 'signer_id': 90, 'source': 'aslsignbank', 'split': 'train', 'url': 'https://aslsignbank.haskins.yale.edu/dictionary/protected_media/glossvideo/ASL/BO/BOOK-418.mp4', 'variation_id': 0, 'video_id': '65225'}, {'bbox': [262, 1, 652, 480], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 2, 'signer_id': 110, 'source': 'valencia-asl', 'split': 'train', 'url': 'https://www.youtube.com/watch?v=0UsjUE-TXns', 'variation_id': 0, 'video_id': '68011'}, {'bbox': [123, 19, 516, 358], 'fps': 25, 'frame_end': 60, 'frame_start': 1, 'instance_id': 3, 'signer_id': 113, 'source': 'lillybauer', 'split'

In [5]:
print("Number of annotations:", len(wlasl_anno))
print("Example entry:", wlasl_anno[0].keys())


Number of annotations: 2000
Example entry: dict_keys(['gloss', 'instances'])


In [6]:
for i in range(3):
    print(wlasl_anno[i]['gloss'], "→", len(wlasl_anno[i]['instances']), "videos")


book → 40 videos
drink → 35 videos
computer → 30 videos


In [7]:
import cv2
import torch
from torch.utils.data import Dataset

class WLASLDataset(Dataset):
    def __init__(self, annotations, root_dir, num_frames=16, transform=None):
        self.annotations = annotations
        self.root_dir = root_dir
        self.num_frames = num_frames
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        label = ann['gloss']   # word label
        video_path = ann['instances'][0]['video_id'] + ".mp4"
        video_fullpath = f"{self.root_dir}/{video_path}"

        cap = cv2.VideoCapture(video_fullpath)
        frames = []
        while len(frames) < self.num_frames and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        cap.release()

        # pad if video shorter
        while len(frames) < self.num_frames:
            frames.append(frames[-1])

        video_tensor = torch.tensor(frames).permute(3,0,1,2) / 255.0  # (C,T,H,W)

        return video_tensor.float(), label


In [8]:
from transformers import TimesformerForVideoClassification

num_classes = len(set([a['gloss'] for a in wlasl_anno]))

model = TimesformerForVideoClassification.from_pretrained(
    "facebook/timesformer-base-finetuned-k400"
)
model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

In [9]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from glob import glob

class VideoDataset(Dataset):
    def __init__(self, root_dir, num_frames=16):
        self.root_dir = root_dir
        self.num_frames = num_frames
        self.samples = []

        # collect all video folders
        for label in os.listdir(root_dir):
            label_path = os.path.join(root_dir, label)
            if os.path.isdir(label_path):
                for vid in os.listdir(label_path):
                    vid_path = os.path.join(label_path, vid)
                    if os.path.isdir(vid_path):
                        self.samples.append((vid_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        frame_paths = sorted(glob(os.path.join(video_path, "*.jpg")))

        # If no frames exist → return dummy tensor
        if len(frame_paths) == 0:
            video_tensor = torch.zeros((3, self.num_frames, 112, 112))
            return video_tensor, label

        frames = []
        for f in frame_paths[:self.num_frames]:
            img = cv2.imread(f)
            if img is not None:
                img = cv2.resize(img, (112,112))
                frames.append(img)

        # If still no valid frames → dummy
        if len(frames) == 0:
            video_tensor = torch.zeros((3, self.num_frames, 112, 112))
            return video_tensor, label

        # Pad with last frame if too short
        while len(frames) < self.num_frames:
            frames.append(frames[-1])

        # Convert to tensor (C, T, H, W)
        video_tensor = torch.tensor(frames).permute(3,0,1,2) / 255.0
        return video_tensor.float(), label


# ✅ Example usage
train_dataset = VideoDataset(root_dir="/content/wlasl-processed/train", num_frames=16)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# 🔍 Debug check
for i in range(3):
    vid, lab = train_dataset[i]
    print(f"Sample {i} → Video shape: {vid.shape}, Label: {lab}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/wlasl-processed/train'

In [10]:
!unzip -n wlasl-processed.zip -d wlasl_data


Archive:  wlasl-processed.zip


In [11]:
import os

print("JSON files:", [f for f in os.listdir("wlasl_data") if f.endswith(".json")][:5])
print("Number of videos:", len(os.listdir("wlasl_data/videos")))


JSON files: ['nslt_300.json', 'nslt_100.json', 'WLASL_v0.3.json', 'nslt_1000.json', 'nslt_2000.json']
Number of videos: 11980


In [ ]:
# =========================================================
# 1. Imports
# =========================================================
import os
import json
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.models.video import r3d_18
from tqdm import tqdm

# =========================================================
# 2. Load WLASL Annotations
# =========================================================
with open("wlasl_data/WLASL_v0.3.json", "r") as f:
    annotations = json.load(f)

video_to_label = {}
for ann in annotations:
    gloss = ann["gloss"]
    for inst in ann["instances"]:
        vid = inst["video_id"] + ".mp4"
        path = os.path.join("wlasl_data/videos", vid)
        if os.path.exists(path):
            video_to_label[path] = gloss

print("Total usable samples:", len(video_to_label))

# =========================================================
# 3. Dataset Class
# =========================================================
class WLASLDataset(Dataset):
    def __init__(self, video_to_label, num_frames=16):
        self.samples = list(video_to_label.items())
        self.num_frames = num_frames
        self.labels = sorted(set(video_to_label.values()))
        self.label2id = {l:i for i,l in enumerate(self.labels)}
        self.id2label = {i:l for l,i in self.label2id.items()}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]

        cap = cv2.VideoCapture(video_path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (112,112))
            frames.append(frame)
        cap.release()

        if len(frames) == 0:
            # fallback if no frames
            video_tensor = torch.zeros((3, self.num_frames, 112, 112))
        else:
            idxs = np.linspace(0, len(frames)-1, self.num_frames).astype(int)
            sampled = [frames[i] for i in idxs]
            video_tensor = torch.tensor(sampled).permute(3,0,1,2) / 255.0

        return video_tensor.float(), self.label2id[label]

# =========================================================
# 4. Create Dataset + Split into Train/Val/Test
# =========================================================
full_dataset = WLASLDataset(video_to_label, num_frames=16)
n_total = len(full_dataset)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [n_train, n_val, n_test])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

# =========================================================
# 5. Load Pretrained Model (3D ResNet)
# =========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(full_dataset.labels)
model = r3d_18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# =========================================================
# 6. Training Loop (Simple)
# =========================================================
EPOCHS = 2  # increase for better training

for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    train_acc = correct / total
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f}")

# =========================================================
# 7. Validation Check
# =========================================================
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)
        outputs = model(X)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)

val_acc = correct / total
print(f"Validation Accuracy: {val_acc:.4f}")

# =========================================================
# 8. Save Model
# =========================================================
torch.save(model.state_dict(), "wlasl_r3d18.pth")
print("✅ Model saved as wlasl_r3d18.pth")


Total usable samples: 11980
Train: 9584, Val: 1198, Test: 1198


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


100%|██████████| 127M/127M [00:00<00:00, 135MB/s]
Epoch 1/2:   0%|          | 0/4792 [00:00<?, ?it/s]/tmp/ipython-input-4083938209.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  video_tensor = torch.tensor(sampled).permute(3,0,1,2) / 255.0
Epoch 1/2:  41%|████      | 1963/4792 [36:10<52:55,  1.12s/it]